In [18]:
# To DO:
# Change format on pop-up

In [44]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
import sys
import string

from pprint import pprint

sys.path.insert(0,'../')
from config import g_key

gmaps.configure(g_key)

In [20]:
vacation_cities_df = pd.read_csv(os.path.join("..","vacation_search","WeatherPy_vacation.csv")).drop(["Unnamed: 0"], axis=1)
vacation_cities_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Lufilufi,WS,-13.8745,-171.5986,78.58,76,47,19.82,light rain,Leuaina Beach Resort and Spa
1,Bredasdorp,ZA,-34.5322,20.0403,61.29,82,96,8.05,overcast clouds,Bredasdorp Country Manor
2,Paris,US,33.6609,-95.5555,71.17,83,75,10.36,broken clouds,"Holiday Inn Express & Suites Paris, an IHG Hotel"
3,Vaini,TO,-21.2000,-175.2000,77.16,100,90,13.80,moderate rain,Keleti Beach Resort
4,Garowe,SO,8.4054,48.4845,69.53,83,8,12.77,clear sky,Curubo Hotel
...,...,...,...,...,...,...,...,...,...,...
224,Santa Maria,CV,16.6000,-22.9000,74.95,74,63,15.17,broken clouds,Porta do Vento
225,Malanje,AO,-9.5402,16.3410,63.12,97,9,1.83,light rain,Hotel Ginga
226,Chimoio,MZ,-19.1164,33.4833,68.92,99,100,3.27,moderate rain,Predio Livraria Ahmed
227,Bandipur,IN,11.6663,76.6286,62.74,96,92,4.07,overcast clouds,OYO 38131 Bandipur Khans Resort


In [21]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

In [26]:
country_code_map_df = pd.read_csv(os.path.join("..","weather_database","country_code_map.csv"))

country_counts_df = pd.DataFrame(data=vacation_cities_df.value_counts("Country"))

country_counts_df.columns = ["City Count"]

eligible_countries_df = pd.merge(left=country_counts_df,
                                 right=country_code_map_df,
                                 left_on = "Country",
                                 right_on = "Code",
                                 how="inner")

eligible_countries_df = eligible_countries_df[["Code", "Name", "City Count"]].loc[(eligible_countries_df["City Count"] >= 4)].set_index("Code")

eligible_countries_df

,Name,City Count
Code,,
ID,Indonesia,20
AU,Australia,20
BR,Brazil,14
US,United States,13
ZA,South Africa,12
NZ,New Zealand,10
IN,India,10
PE,Peru,9
MX,Mexico,8


In [36]:
# Allow user to choose country.
chosen_country_code = input("Please choose a country by entering the CODE value from above list of eligible countries: ").upper()

while len(eligible_countries_df.loc[eligible_countries_df.index == chosen_country_code]) == 0:
    chosen_country_code = input("Invalid selection.  Please try again: ").upper()

chosen_country_name = eligible_countries_df.loc[eligible_countries_df.index == chosen_country_code]["Name"].get(key=0)
chosen_country_count = eligible_countries_df.loc[eligible_countries_df.index == chosen_country_code]["City Count"].get(key=0)

print(f"You have chosen {chosen_country_name}, which has {chosen_country_count} eligible cities: ")

chosen_country_cities_df = vacation_cities_df.loc[vacation_cities_df["Country"] == chosen_country_code]
chosen_country_cities_df.reset_index(drop=True, inplace=True)
chosen_country_cities_df

Please choose a country code using the above list of eligible countries:  IN


You have chosen India, which has 10 eligible cities: 


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Kanniyakumari,IN,8.0833,77.5667,77.45,80,6,19.91,clear sky,Sparsa
1,Akola,IN,20.7333,77.0000,64.15,66,5,7.25,clear sky,Hotel City Pride
2,Yarada,IN,17.6500,83.2667,71.92,83,20,2.30,mist,Yarada Jungle Beach Resorts
3,Digras,IN,20.1167,77.7167,62.65,71,0,3.98,clear sky,Swad Restaurant
4,Kankon,IN,15.0167,74.0167,76.80,66,25,4.88,scattered clouds,Palolem Beach Resort
5,Shahpur,IN,16.7000,76.8333,68.47,70,54,5.64,broken clouds,Ashwini Hotel & Deluxe Lodge
6,Dwarka,IN,22.2394,68.9678,69.49,56,2,15.05,clear sky,Hotel Narayan Avezika Comfort
7,Radhanpur,IN,23.8333,71.6000,60.64,52,19,7.47,few clouds,"HONEST - HOTEL (VIRAT HOTEL AND GUEST HOUSE, R..."
8,Malpe,IN,13.3500,74.7167,78.10,73,33,7.49,scattered clouds,Hotel Sri Krishna Residency
9,Bandipur,IN,11.6663,76.6286,62.74,96,92,4.07,overcast clouds,OYO 38131 Bandipur Khans Resort


In [57]:
# city decision engine
    # pick cities:
        # Allow user to pick city (cities?) based on list
        # Regenerate df by adding column and sorting by distance.  Consider using .pop()
    # map using optimize route

In [85]:
chosen_city_ids = list()

city_id_1 = int(input("Using the number value on the far left, please choose a starting/ending city for your trip: "))

while len(chosen_country_cities_df.loc[chosen_country_cities_df.index == city_id_1]) == 0:
    city_id_1 = int(input("Invalid selection.  Please try again: "))
    
chosen_city_ids.append(city_id_1)
    
vacation_start = chosen_country_cities_df.iloc[city_id_1]
vacation_end = chosen_country_cities_df.iloc[city_id_1]
city = vacation_start["City"]

print(f"You have chosen {city}.")

vacation_start

for i in range(3):
    city_id = int(input(f"Please identify city #{i+2} on your itinerary: "))
    
    while ((len(chosen_country_cities_df.loc[chosen_country_cities_df.index == city_id]) == 0) or (city_id in chosen_city_ids)):
        if (city_id in chosen_city_ids):
            temp_df = chosen_country_cities_df.iloc[city_id]
            city_name = temp_df["City"]
            city_id = int(input(f"You have already chosen {city_name}.  Please try again: "))
        else:
            city_id = int(input("Invalid selection.  Please try again: "))

    chosen_city_ids.append(city_id)    
    
    temp_df = chosen_country_cities_df.iloc[city_id]
    city_name = temp_df["City"]
    print(f"You have chosen {city_name}.")
        
    # This is ugly and ought to be refactored.
    if (i + 2 == 2):
        vacation_stop1 = chosen_country_cities_df.iloc[city_id]
    elif (i + 2 == 3):
        vacation_stop2 = chosen_country_cities_df.iloc[city_id]    
    elif (i + 2 == 4):
        vacation_stop3 = chosen_country_cities_df.iloc[city_id]    
    
# # convert each chosen city to a dataframe
# vacation_start = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_1)]
# vacation_end = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_1)]
# vacation_stop1 = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_2)]
# vacation_stop2 = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_3)]
# vacation_stop3 = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_4)]

vacation_start

Using the number value on the far left, please choose a starting/ending city for your trip:  3


You have chosen Digras.


Please identify city #2 on your itinerary:  55
Invalid selection.  Please try again:  4


You have chosen Kankon.


Please identify city #3 on your itinerary:  5


You have chosen Shahpur.


Please identify city #4 on your itinerary:  9


You have chosen Bandipur.


City                       Digras
Country                        IN
Lat                       20.1167
Long                      77.7167
Max Temp                    62.65
Pct Humidity                   71
Pct Cloudiness                  0
Wind Speed                   3.98
Description             clear sky
Hotel Name        Swad Restaurant
Name: 3, dtype: object

In [89]:
# Get the lat-long pairs
start = vacation_start[["Lat", "Long"]].to_numpy()
end = vacation_end[["Lat", "Long"]].to_numpy()
stop1 = vacation_stop1[["Lat", "Long"]].to_numpy()
stop2 = vacation_stop2[["Lat", "Long"]].to_numpy()
stop3 = vacation_stop3[["Lat", "Long"]].to_numpy()

In [90]:
fig = gmaps.figure()
chosen_trip = gmaps.directions_layer(
         start=(start[0],start[1]),
         end=(end[0],end[1]),
         waypoints=[(stop1[0],stop1[1]),
                    (stop2[0],stop2[1]),
                    (stop3[0],stop3[1])],
        optimize_waypoints=True
)
fig.add_layer(chosen_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [91]:
itinerary_df = pd.concat(objs=[vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],
                         ignore_index=True)
itinerary_df

0                              Digras
1                                  IN
2                             20.1167
3                             77.7167
4                               62.65
5                                  71
6                                   0
7                                3.98
8                           clear sky
9                     Swad Restaurant
10                             Kankon
11                                 IN
12                            15.0167
13                            74.0167
14                               76.8
15                                 66
16                                 25
17                               4.88
18                   scattered clouds
19               Palolem Beach Resort
20                            Shahpur
21                                 IN
22                               16.7
23                            76.8333
24                              68.47
25                                 70
26          

In [92]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

coordinates_df = itinerary_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure( #center=(0.0, 0.0), 
                   # zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

AttributeError: 'Series' object has no attribute 'iterrows'